[![Generic badge](https://img.shields.io/badge/License-Properitary-red.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/blob/master/LICENSE.md) <br>
[![Active Development](https://img.shields.io/badge/Maintenance%20Level-Actively%20Developed-brightgreen.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/) <br>
[![Ask Me Anything !](https://img.shields.io/badge/Ask%20me-anything-1abc9c.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/issues) <br>
[![Python 3.7.7](https://img.shields.io/badge/python-3.7.7-blue.svg)](https://www.python.org/downloads/release/python-377/)

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Logo_fh_hof.svg/2000px-Logo_fh_hof.svg.png" width="300" style="background-color:#FFF">

<font size="+4"><i><u>Q-Learning mit 'Super Mario Bros'</u></i></font>

Seminararbeit der Vorlesung **Angewandtes Maschinelles Lernen** an der **Hochschule für angewande Wissenschaften Hof** des **Sommersemesters 2020**.

# Vorbereitungen

*Code der das Jupyter-Notebook vorbereit.*

## Notebook-Styling

*Ändert u.a. das Layout des Notebooks.*

In [1]:
from IPython.core.display import display, HTML

_notebook_relative_width = "95%" # Wertebereich: 0-100, muss nach HTML-Schema definiert sein
display(HTML('<style>.container { width: ' + _notebook_relative_width + ' !important; }</style>'))

## Requirements

*Die benötigten Packages wurden aus der bei mir installierten Pip-Liste gebildet.*

**Hinweis:** *Es werden* ***mehr als zwingend notwendige*** *Packages installiert.*

In [2]:
# Package-Liste
#!pip list

In [3]:
%%capture
# Installation
#!pip install -r requirements.txt

In [4]:
%%capture
# Update 
#!pip install -r requirements.txt --upgrade

In [5]:
%%capture
# Export Pip-Packages nach './requirements.txt'
#!pip freeze > requirements.txt

# Konfiguration

*Import-Statements & Vorbereitung der Frameworks/Module zusammengefasst.*

## Allgemein

*Nicht weiter spezifizierte Import-Statements, bspw. von System-Bibliotheken.*

In [6]:
import datetime, os

In [7]:
# Erstelle Logs-Folder falls nicht existent.
logs_base_dir = "./logs"
if not os.path.isdir(logs_base_dir):
    !mkdir logs

## Tensorflow

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1280px-TensorFlowLogo.svg.png" width="150px"/>

*TensorFlow ist ein Framework zur datenstromorientierten Programmierung. Es wird aus Python-Programmen heraus benutzt und ist in Python und C++ implementiert. Populäre Anwendung findet TensorFlow im Bereich des maschinellen Lernens. [...]*

[Weiter Informationen](https://pypi.org/project/tensorflow/)

In [8]:
import tensorflow as tf

## Tensorboard

<img src="https://www.tensorflow.org/site-assets/images/project-logos/tensorboard-logo-social.png" width="250px"/>

TensorBoard ist ein Sammlung von Web-Applicationen um TensorFlow zu inspizieren und zu verstehen.

[Weitere Informationen](https://pypi.org/project/tensorboard/)

In [9]:
%load_ext tensorboard

In [16]:
%%capture

from tensorboard import notebook

# Starten einer Tensorboard-Instanz mit Logs & gebindeten Ports
%tensorboard --port=6007 --logdir {logs_base_dir} --bind_all

# Auflisten der TensorBoard-Instanzen (resourcen intesiv)
notebook.list()

print("\nYou can manually connect to it via Browser using this host-ip at port 6007.")

## OpenAI-Gym

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/OpenAI_Logo.svg/1920px-OpenAI_Logo.svg.png" width="275px"/>

*OpenAI-Gym ist ein Toolkit für das Entwickeln und Vergleichen von Reinforcement-Learning-Algorithmen.*

[Weitere Informationen](https://pypi.org/project/gym/)

In [17]:
import gym

from gym.wrappers import Monitor
from gym import logger as gymlogger

# Nur Error-Logging
gymlogger.set_level(40)

## Gym-Super-Mario-Bros

*Eine OpenAI-Gym-Umgebung für Super Mario Bros. und Super Mario Bros. 2 (Lost Levels) auf den NES (Nitendo Entertainment System) mithilfe des nes-py emulators.*

[Weitere Informationen](https://pypi.org/project/gym-super-mario-bros/)

In [18]:
import gym_super_mario_bros

from nes_py.wrappers import JoypadSpace # nes_py is a direct dependency by gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY, SIMPLE_MOVEMENT, COMPLEX_MOVEMENT 

## Stable-Baselines

Stable Baselines ist eine Sammlung von optimierten Implementation von Reinforcement-Learning-Algorithmen basierend auf OpenAi.

[Weitere Informationen](https://pypi.org/project/stable-baselines/)

In [ ]:
# No imports-yet